In [1]:
from transformers import AutoProcessor, AutoModel
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
text = "a cat."

processor = AutoProcessor.from_pretrained("IDEA-Research/grounding-dino-tiny")
model = AutoModel.from_pretrained("IDEA-Research/grounding-dino-tiny")

inputs = processor(images=image, text=text, return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state


/Users/edbayes/.pyenv/versions/3.10.0/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[1, 900, 256]

In [4]:
print(last_hidden_states)
#list(last_hidden_states.shape)

# input_ids, 
init_reference_points = outputs.init_reference_points
print(init_reference_points)



tensor([[[0.7678, 0.4134, 0.4555, 0.7304],
         [0.2555, 0.5436, 0.4788, 0.8742],
         [0.2551, 0.5468, 0.4783, 0.8792],
         ...,
         [0.7011, 0.9686, 0.1338, 0.0750],
         [0.3809, 0.2748, 0.3417, 0.3963],
         [0.8871, 0.0227, 0.1271, 0.0597]]])


In [2]:
import torch
from transformers import AutoProcessor, GroundingDinoForObjectDetection
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
flower = "./input/below_canopy/DJI_0017.JPG"
# image = Image.open(requests.get(url, stream=True).raw)
image = Image.open(flower)
text = "flower. plant. fern. palm. berries."

processor = AutoProcessor.from_pretrained("IDEA-Research/grounding-dino-tiny")
model = GroundingDinoForObjectDetection.from_pretrained("IDEA-Research/grounding-dino-tiny")

inputs = processor(images=image, text=text, return_tensors="pt")
outputs = model(**inputs)

# convert outputs (bounding boxes and class logits) to COCO API
target_sizes = torch.tensor([image.size[::-1]])
results = processor.image_processor.post_process_object_detection(
    outputs, threshold=0.35, target_sizes=target_sizes
)[0]
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 1) for i in box.tolist()]
    print(f"Detected {label.item()} with confidence " f"{round(score.item(), 2)} at location {box}")

Detected 1 with confidence 0.61 at location [1835.7, 714.3, 2279.8, 1067.1]
Detected 1 with confidence 0.57 at location [2287.8, 1258.3, 2849.4, 1675.5]
